In [36]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib notebook

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Conv1D, MaxPooling1D, Reshape, Input

# Make simple feed-forward models
for just timeseries or just FFT data

In [63]:
timeseries_input_shape = (4000,)

timeseries_feedforward_model = Sequential([
    Dense(1028, input_shape=timeseries_input_shape),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(16),
    Activation('relu'),
    Dense(2),
    Activation('softmax'),
])

timeseries_feedforward_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_131 (Dense)            (None, 1028)              4113028   
_________________________________________________________________
activation_95 (Activation)   (None, 1028)              0         
_________________________________________________________________
dense_132 (Dense)            (None, 64)                65856     
_________________________________________________________________
activation_96 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_133 (Dense)            (None, 16)                1040      
_________________________________________________________________
activation_97 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_134 (Dense)            (None, 2)                 34        
__________

In [64]:
fft_input_shape = (2001,2)

fft_feedforward_model = Sequential([
    Flatten(input_shape=fft_input_shape),
    Dense(1028),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(16),
    Activation('relu'),
    Dense(2),
    Activation('softmax'),
])

fft_feedforward_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_27 (Flatten)         (None, 4002)              0         
_________________________________________________________________
dense_135 (Dense)            (None, 1028)              4115084   
_________________________________________________________________
activation_99 (Activation)   (None, 1028)              0         
_________________________________________________________________
dense_136 (Dense)            (None, 64)                65856     
_________________________________________________________________
activation_100 (Activation)  (None, 64)                0         
_________________________________________________________________
dense_137 (Dense)            (None, 16)                1040      
_________________________________________________________________
activation_101 (Activation)  (None, 16)                0         
__________

# Make convolutional models
for just timeseries or just FFT data

In [55]:
timeseries_input_shape = (4000,)

timeseries_convolutional_model = Sequential([
    Reshape((4000,1), input_shape=timeseries_input_shape),
    Conv1D(filters=64, kernel_size=16, padding='same'),
    Activation('relu'),
    MaxPooling1D(pool_size=8),
    Conv1D(filters=16, kernel_size=16, padding='same'),
    Activation('relu'),
    MaxPooling1D(pool_size=8),
    Conv1D(filters=4, kernel_size=16, padding='same'),
    Activation('relu'),
    MaxPooling1D(pool_size=8),
    Flatten(),
    Dense(2),
    Activation('softmax'),
])

timeseries_convolutional_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_8 (Reshape)          (None, 4000, 1)           0         
_________________________________________________________________
conv1d_52 (Conv1D)           (None, 4000, 64)          1088      
_________________________________________________________________
activation_79 (Activation)   (None, 4000, 64)          0         
_________________________________________________________________
max_pooling1d_33 (MaxPooling (None, 500, 64)           0         
_________________________________________________________________
conv1d_53 (Conv1D)           (None, 500, 16)           16400     
_________________________________________________________________
activation_80 (Activation)   (None, 500, 16)           0         
_________________________________________________________________
max_pooling1d_34 (MaxPooling (None, 62, 16)            0         
__________

In [57]:
fft_input_shape = (2001,2)

fft_convolutional_model = Sequential([
    Conv1D(filters=64, kernel_size=16, padding='same', input_shape=fft_input_shape),
    Activation('relu'),
    MaxPooling1D(pool_size=8),
    Conv1D(filters=16, kernel_size=16, padding='same'),
    Activation('relu'),
    MaxPooling1D(pool_size=8),
    Conv1D(filters=4, kernel_size=16, padding='same'),
    Activation('relu'),
    MaxPooling1D(pool_size=8),
    Flatten(),
    Dense(2),
    Activation('softmax'),
])

fft_convolutional_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_58 (Conv1D)           (None, 2001, 64)          2112      
_________________________________________________________________
activation_87 (Activation)   (None, 2001, 64)          0         
_________________________________________________________________
max_pooling1d_38 (MaxPooling (None, 250, 64)           0         
_________________________________________________________________
conv1d_59 (Conv1D)           (None, 250, 16)           16400     
_________________________________________________________________
activation_88 (Activation)   (None, 250, 16)           0         
_________________________________________________________________
max_pooling1d_39 (MaxPooling (None, 31, 16)            0         
_________________________________________________________________
conv1d_60 (Conv1D)           (None, 31, 4)             1028      
__________

# Make Combined Models
This requires using the Keras functional API as opposed to the sequential API

In [60]:
timeseries_input_shape = (4000,)
fft_input_shape = (2001,2)

timeseries_input = Input(shape=timeseries_input_shape)
fft_input = Input(shape=fft_input_shape)

timeseries_model = Dense(1028, activation='relu')(timeseries_input)
timeseries_model = Dense(64, activation='relu')(timeseries_model)
timeseries_model = Dense(16, activation='relu')(timeseries_model)

fft_model = Flatten()(fft_input)
fft_model = Dense(1028, activation='relu')(fft_model)
fft_model = Dense(64, activation='relu')(fft_model)
fft_model = Dense(16, activation='relu')(fft_model)

shared_output = keras.layers.concatenate([timeseries_model, fft_model], axis=-1)
shared_output = Dense(8, activation='relu')(shared_output)
shared_output = Dense(2, activation='softmax')(shared_output)

split_feedforward_model = Model(inputs=[timeseries_input, fft_input], outputs=shared_output)

split_feedforward_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_34 (InputLayer)            (None, 2001, 2)       0                                            
____________________________________________________________________________________________________
input_33 (InputLayer)            (None, 4000)          0                                            
____________________________________________________________________________________________________
flatten_24 (Flatten)             (None, 4002)          0           input_34[0][0]                   
____________________________________________________________________________________________________
dense_117 (Dense)                (None, 1028)          4113028     input_33[0][0]                   
___________________________________________________________________________________________

In [61]:
timeseries_input_shape = (4000,)
fft_input_shape = (2001,2)

timeseries_input = Input(shape=timeseries_input_shape)
fft_input = Input(shape=fft_input_shape)

timeseries_model = Reshape((4000,1))(timeseries_input)
timeseries_model = Conv1D(filters=64, kernel_size=16, padding='same', activation='relu')(timeseries_model)
timeseries_model = MaxPooling1D(pool_size=8)(timeseries_model)
timeseries_model = Conv1D(filters=16, kernel_size=16, padding='same', activation='relu')(timeseries_model)
timeseries_model = MaxPooling1D(pool_size=8)(timeseries_model)
timeseries_model = Conv1D(filters=4, kernel_size=16, padding='same', activation='relu')(timeseries_model)
timeseries_model = MaxPooling1D(pool_size=8)(timeseries_model)
timeseries_model = Flatten()(timeseries_model)

fft_model = Conv1D(filters=64, kernel_size=16, padding='same', activation='relu')(fft_input)
fft_model = MaxPooling1D(pool_size=8)(fft_model)
fft_model = Conv1D(filters=16, kernel_size=16, padding='same', activation='relu')(fft_model)
fft_model = MaxPooling1D(pool_size=8)(fft_model)
fft_model = Conv1D(filters=4, kernel_size=16, padding='same', activation='relu')(fft_model)
fft_model = MaxPooling1D(pool_size=8)(fft_model)
fft_model = Flatten()(fft_model)

shared_output = keras.layers.concatenate([timeseries_model, fft_model], axis=-1)
shared_output = Dense(16, activation='relu')(shared_output)
shared_output = Dense(2, activation='softmax')(shared_output)

split_convolutional_model = Model(inputs=[timeseries_input, fft_input], outputs=shared_output)

split_convolutional_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_35 (InputLayer)            (None, 4000)          0                                            
____________________________________________________________________________________________________
reshape_11 (Reshape)             (None, 4000, 1)       0           input_35[0][0]                   
____________________________________________________________________________________________________
input_36 (InputLayer)            (None, 2001, 2)       0                                            
____________________________________________________________________________________________________
conv1d_73 (Conv1D)               (None, 4000, 64)      1088        reshape_11[0][0]                 
___________________________________________________________________________________________

# Save all the models

In [65]:
with open('timeseries_feedforward_model.json', 'w') as text_file:
    text_file.write(timeseries_feedforward_model.to_json())

with open('fft_feedforward_model.json', 'w') as text_file:
    text_file.write(fft_feedforward_model.to_json())

with open('split_feedforward_model.json', 'w') as text_file:
    text_file.write(split_feedforward_model.to_json())

with open('timeseries_convolutional_model.json', 'w') as text_file:
    text_file.write(timeseries_convolutional_model.to_json())

with open('fft_convolutional_model.json', 'w') as text_file:
    text_file.write(fft_convolutional_model.to_json())

with open('split_convolutional_model.json', 'w') as text_file:
    text_file.write(split_convolutional_model.to_json())